## Lowess filtering

In [1]:
import os, sys
import pickle
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pyvital2 import arr

# 0.2가 제일 잘 없앴음
def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T

In [8]:
import os, sys
import pickle
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 0.2가 제일 잘 없앴음
def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T


LEN_INPUT = 20
OVERLAP = 10
SRATE = 100
LEN_PER_POST = 120
n_aug = int((60-LEN_INPUT)/OVERLAP) + 1  
n_aug2 = int((LEN_PER_POST-LEN_INPUT)/OVERLAP) + 1


df_preprocess = pickle.load(open('cache/preprocess/df_preprocess', 'rb'))
for f_num, rows in df_preprocess.iterrows():
    caseid = rows['caseid']
    print(f'\n###Input{f_num}/{len(df_preprocess)}: {caseid}###')
    

    # vital data 불러오기    
    file_path = 'input_60s'
    vals = np.load(f'{file_path}/{caseid}.npz')


    # 20초 단위로 끊기
    for i in range(n_aug):
        print('  n_segment', i+1, end='')
        start_idx = i*OVERLAP*SRATE # 500i
        end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000
        
        # non-event data
        if rows[str(i+1)][0]:
            print(' passed...lowess filtering...', end='')
            
            save_path = f'cache/lowess_filtered/input20s-10s/{caseid}_n{i+1}.npz'
            if os.path.exists(save_path):
                print('already exists', end='')
                continue

            ppg_inp = vals['nPPG'][start_idx:end_idx]
            ecg_inp = vals['nECG'][start_idx:end_idx]
            
            ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
            ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
            
            # lowess filter 적용
            ppg_input = ppg_inp - lowess(ppg_inp)
            ecg_input = ecg_inp - lowess(ecg_inp)
            
            np.savez(save_path, ECG = ecg_input, PPG = ppg_input)
            print('done', end=' ')

    print('')
    for i in range(n_aug):
        print('  segment', i+1, end='')
        start_idx = i*OVERLAP*SRATE # 500i
        end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000
            
        # event data
        if rows[str(i+n_aug+1)][0]:
            print(' passed...lowess filtering...', end='')
            
            save_path = f'cache/lowess_filtered/input20s-10s/{caseid}_{i+1}.npz'
            if os.path.exists(save_path):
                print('already exists', end='')
                continue

            ppg_inp = vals['PPG'][start_idx:end_idx]
            ecg_inp = vals['ECG'][start_idx:end_idx]
            
            ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
            ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
            
            # lowess filter 적용
            ppg_input = ppg_inp - lowess(ppg_inp)
            ecg_input = ecg_inp - lowess(ecg_inp)
            
            np.savez(save_path, ECG = ecg_input, PPG = ppg_input) 
            print('done', end=' ')


###Input0/2684: 5974###
  n_segment 1 passed...lowess filtering...already exists  n_segment 2 passed...lowess filtering...already exists  n_segment 3 passed...lowess filtering...already exists  n_segment 4 passed...lowess filtering...already exists  n_segment 5 passed...lowess filtering...already exists
  segment 1 passed...lowess filtering...already exists  segment 2 passed...lowess filtering...already exists  segment 3 passed...lowess filtering...already exists  segment 4 passed...lowess filtering...already exists  segment 5 passed...lowess filtering...already exists
###Input1/2684: 801###
  n_segment 1 passed...lowess filtering...already exists  n_segment 2 passed...lowess filtering...already exists  n_segment 3 passed...lowess filtering...already exists  n_segment 4 passed...lowess filtering...already exists  n_segment 5 passed...lowess filtering...already exists
  segment 1  segment 2  segment 3  segment 4  segment 5
###Input2/2684: 2790###
  n_segment 1 passed...lowess filtering

In [13]:
import psutil

psutil.cpu_percent()

ModuleNotFoundError: No module named 'psutil'

In [20]:
#%load_ext autoreload
#%autoreload 2

import learn

learn.load_vital_data('vital_to_np')

loading vital data...
1/2699(3)...already existing
2/2699(8)...already existing
3/2699(9)...already existing
4/2699(13)...already existing
5/2699(14)...already existing
6/2699(16)...already existing
7/2699(17)...already existing
8/2699(19)...already existing
9/2699(20)...already existing
10/2699(22)...already existing
11/2699(28)...already existing
12/2699(29)...already existing
13/2699(30)...already existing
14/2699(35)...already existing
15/2699(36)...already existing
16/2699(40)...already existing
17/2699(42)...already existing
18/2699(44)...already existing
19/2699(45)...already existing
20/2699(46)...already existing
21/2699(47)...already existing
22/2699(48)...already existing
23/2699(50)...already existing
24/2699(54)...already existing
25/2699(57)...already existing
26/2699(59)...already existing
27/2699(67)...already existing
28/2699(69)...already existing
29/2699(70)...already existing
30/2699(72)...already existing
31/2699(74)...already existing
32/2699(75)...already existin

/home/painstudy/anaconda3/envs/keras/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


ValueError: Must have equal len keys and value when setting with an iterable

In [21]:
#%load_ext autoreload
#%autoreload 2
import learn
import pickle

learn.preprocess(file_path='vital_to_np', LEN_INPUT = 20, OVERLAP = 10, SRATE = 100)

loading vital data...
1/2699(3)...already existing
2/2699(8)...already existing
3/2699(9)...already existing
4/2699(13)...already existing
5/2699(14)...already existing
6/2699(16)...already existing
7/2699(17)...already existing
8/2699(19)...already existing
9/2699(20)...already existing
10/2699(22)...already existing
11/2699(28)...already existing
12/2699(29)...already existing
13/2699(30)...already existing
14/2699(35)...already existing
15/2699(36)...already existing
16/2699(40)...already existing
17/2699(42)...already existing
18/2699(44)...already existing
19/2699(45)...already existing
20/2699(46)...already existing
21/2699(47)...already existing
22/2699(48)...already existing
23/2699(50)...already existing
24/2699(54)...already existing
25/2699(57)...already existing
26/2699(59)...already existing
27/2699(67)...already existing
28/2699(69)...already existing
29/2699(70)...already existing
30/2699(72)...already existing
31/2699(74)...already existing
32/2699(75)...already existin

ValueError: Must have equal len keys and value when setting with an iterable

## intubation time

In [14]:
import vitaldb
import pandas


df = pd.read_csv("https://api.vitaldb.net/cases")
caseids = list(vitaldb.caseids_tiva & set(df.loc[df['ane_type'] == 'General', 'caseid']))
track_names = ["SNUADC/ECG_II", "SNUADC/PLETH", "Solar8000/VENT_INSP_TM", "Primus/SET_INSP_TM", "Orchestra/PPF20_CE", "Orchestra/RFTN20_CE"]

if not os.path.exists(file_path):
    os.mkdir(file_path)

cnt = 0
for caseid in caseids[:1]:
    cnt = cnt + 1
    print(f'{cnt}/{len(caseids)}({caseid})', end='...')

    # check if file is already existing
    filename = f'{file_path}/{caseid}.npz'
    if os.path.isfile(filename):
        print('already existing')
        continue


    # get vital file and save as numpy
    vf = vitaldb.VitalFile(caseid, track_names)
    vals = vf.to_numpy(track_names, interval=1/SRATE)


    # intubation time - find the first t which satisfies vent_set_tm != nan & ppf_ce != nan
    t_intu = np.where(~np.isnan(vals[:,5]))[0][0]

    if not np.mean(~np.isnan(vals[:,2])):
        if not np.mean(~np.isnan(vals[:,3])):
            print(f'caseid {caseid} - no valid data for insp_tm')
            continue
        intu = vals[:,3]
        intv = 850 # maximum interval for "Primus/SET_INSP_TM"
    else:
        intu = vals[:,2]
        intv = 250 # maximum interval for "Solar8000/VENT_INSP_TM"

    idc_intu = np.where(~np.isnan(intu))[0]
    while True:
        # vent_insp_tm이 nan이 아닌 경우
        if not np.isnan(intu[t_intu]):
            print(t_intu, end=' ')
            idx = np.where(idc_intu==t_intu)[0][0]
            prev = t_intu

            switch = True
            for i in range(1,11):
                if idc_intu[idx+i] - prev > intv:
                    switch = False
                    t_intu = t_intu + 1
                prev = idc_intu[idx+i]
            if switch:
                break
        else:
            t_intu = t_intu + 1


    # non-event data : extract vital from previous 120s-60s from intubation
    ppg = vals[:,1]
    prev_ppg = ppg[t_intu - SRATE*120:t_intu - SRATE*60]

    ecg = vals[:,0]
    prev_ecg = ecg[t_intu - SRATE*120:t_intu - SRATE*60]


    # after intubation, pain calculated using TSS, CISA
    post_ppg = ppg[t_intu:t_intu + SRATE*LEN_PER_POST]
    post_ecg = ecg[t_intu:t_intu + SRATE*LEN_PER_POST]

    ppf = vals[:,4]
    ppf = ppf[t_intu:t_intu + SRATE*LEN_PER_POST]

    rftn = vals[:,5]
    rftn = rftn[t_intu:t_intu + SRATE*LEN_PER_POST]


    np.savez(filename, nECG=prev_ecg, nPPG=prev_ppg, ECG=post_ecg, PPG=post_ppg, PPF=ppf, RFTN=rftn)

1/2699(3)...48557 

## preprocessing

In [33]:
file_path = 'vital_to_np'
LEN_INPUT = 20
OVERLAP = 10
SRATE = 100
LEN_PER_POST = 120

def linear_connection(list, idx):
    int_idx = int(idx)
    return list[int_idx] + (list[int_idx+1] - list[int_idx]) * (idx - int_idx)


# path for cache
if not os.path.exists('./cache'):
    os.mkdir('./cache')
if not os.path.exists('./cache/peaks'):
    os.mkdir('./cache/peaks')
if not os.path.exists(f"cache/peaks/PPG_{SRATE}Hz_1min_seg"):
    os.mkdir(f"cache/peaks/PPG_{SRATE}Hz_1min_seg")
if not os.path.exists(f"cache/peaks/ECG_{SRATE}Hz_1min_seg"):
    os.mkdir(f"cache/peaks/ECG_{SRATE}Hz_1min_seg")        
if not os.path.exists('./cache/preprocess'):
    os.mkdir('./cache/preprocess')


# dataframe to save preprocessing info
n_aug = int((60-LEN_INPUT)/OVERLAP) + 1   # number of data augmentation
n_aug2 = int((LEN_PER_POST-LEN_INPUT)/OVERLAP) + 1
column_list = ['caseid'] + [str(i+1) for i in range(n_aug+n_aug2)]
df_preprocess = pd.DataFrame(columns = column_list)


# set variables
caseids = os.listdir(file_path)
error_list = []
f_num = 0
initial = f_num
interval = 1 #len(caseids)


for caseid in caseids[initial:initial+interval]:
    caseid = caseid[:-4]
    f_num += 1
    print('\n###Input', f_num,'/ '+str(len(caseids))+': '+caseid+'###')


    # vital data 불러오기    
    vals = np.load(f'{file_path}/{caseid}.npz')


    #dataframe에 새로운 행 만들기
    df_preprocess.loc[f_num-1,'caseid'] = caseid

    ppg_cache = f"cache/peaks/PPG_{SRATE}Hz_1min_seg/" + caseid
    ecg_cache = f"cache/peaks/ECG_{SRATE}Hz_1min_seg/" + caseid    
    ecg_cache2 = f"cache/peaks/ECG_{SRATE}Hz_1min_seg/" + caseid


    # 20초 단위로 끊기
    for i in range(n_aug):
        print('  segment', i+1, end='')
        start_idx = i*OVERLAP*SRATE # 500i
        end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000


        ### non-event input ###
        seg_ppg = vals['nPPG'][start_idx:end_idx]
        seg_ecg = vals['nECG'][start_idx:end_idx]


        ## 1. 결측치 처리 ##             
        # df.isnull().sum() 하면 더 간단하게 가능하나 애초에 NRS에 해당하는 vital data가 120초 보다 짧은 경우
        nan_ppg_list = np.isnan(seg_ppg)
        nan_ecg_list = np.isnan(seg_ecg)
        nan_ppg_perc = np.sum(nan_ppg_list) / LEN_INPUT / SRATE
        nan_ecg_perc = np.sum(nan_ecg_list) / LEN_INPUT / SRATE

        # ECG, PPG 둘다 결측치인 부분
        nan_both_perc = 0
        for j in range(len(seg_ppg)):
            if nan_ppg_list[j] and  nan_ecg_list[j]:
                nan_both_perc += 1
        nan_both_perc /= (LEN_INPUT*SRATE)

        # segment의 결측치 비율 정보
        nan_info = [nan_ppg_perc, nan_ecg_perc, nan_both_perc]

        # 결측치가 많은 경우, noise 확인할 것도 없이 False -  이 경우의 noise_info는 -1로 처리
        if nan_ppg_perc > 0.05 or nan_ecg_perc > 0.05 or nan_both_perc > 0.05:
            df_preprocess.loc[f_num-1,str(i+1)] = (False, nan_info, [-1, -1])
            print('too much missing data')
            continue


        ## 2. Noise 처리 ##
        # peak detection
        if os.path.exists(ppg_cache+'_n{}'.format(i+1)):
            _, ppg_peak = pickle.load(open(ppg_cache+'_n{}'.format(i+1), 'rb'))
            ecg_peak = pickle.load(open(ecg_cache+'_n{}'.format(i+1), 'rb'))
            print('...loaded peak...', end='')


        else:
            try:
                min_peak, ppg_peak = arr.detect_peaks(pd.DataFrame(seg_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)
                ecg_peak = arr.detect_qrs(pd.DataFrame(seg_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)


            except Exception as e:
                print('error of', e)
                error_list.append(caseid)
                df_preprocess.loc[f_num-1,str(i+1)] = (False, nan_info, [-3, -3])
                continue


            if len(ppg_peak)==0:
                print('no peak')


            pickle.dump((min_peak, ppg_peak), open(ppg_cache+'_n{}'.format(i+1), 'wb'))
            pickle.dump(ecg_peak, open(ecg_cache+'_n{}'.format(i+1), 'wb'))
            print('...saved peak...', end='')


        # 10초 segment 내의 ppg, ecg peak idx
        #seg_ppg_min = ppg_min[(start_idx<=np.array(ppg_min)) & (np.array(ppg_min)<end_idx)]
        idx_ppg_peak = ppg_peak
        idx_ecg_peak = ecg_peak


        # peak가 HR 30~150 -> 20s - min 10 peaks(HR30)
        # peak 개수가 기준 미달이면 noise 계산 자세히 할 필요없이 False - 이 경우의 noise_info는 -2로 처리
        if len(idx_ppg_peak)<5/10*LEN_INPUT or len(idx_ecg_peak)<5/10*LEN_INPUT:
            df_preprocess.loc[f_num-1,str(i+1)] = (False, nan_info, [-2, -2])
            print('too less peaks')
            continue


        # 20초 segment 내의 ppg, ecg peak value
        #print(len(seg_ppg), idx_ppg_peak)
        val_ppg_peak = [seg_ppg[k] for k in idx_ppg_peak]
        val_ecg_peak = [seg_ecg[k] for k in idx_ecg_peak]

        # peak와 peak 사이 interval에 대한 noise 여부 -> 따라서 길이는 peak - 1
        bool_noise_ppg = [False for k in range(len(idx_ppg_peak)-1)]
        bool_noise_ecg = [False for k in range(len(idx_ecg_peak)-1)]


        #  2.1 peak 간격 이상한 noise (HR 30~150 -> HBI 0.4s ~ 2s로 SRATE 곱해주면 40~200)
        for k in range(len(bool_noise_ppg)):
            if not 0.4*SRATE < idx_ppg_peak[k+1] - idx_ppg_peak[k] < 2*SRATE:
                bool_noise_ppg[k] = True
        for k in range(len(bool_noise_ecg)):
            if not 0.4*SRATE < idx_ecg_peak[k+1] - idx_ecg_peak[k] < 2*SRATE:
                bool_noise_ecg[k] = True


        # 2.2 모양 이상한 noise
        # wave interval into same length(2s(200))
        len_wave = 2*SRATE
        norm_seg_ppg, norm_seg_ecg = [], []

        for k in range(len(bool_noise_ppg)):
            len_interval_ppg = idx_ppg_peak[k+1] - idx_ppg_peak[k]

            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ppg.append([linear_connection(seg_ppg[idx_ppg_peak[k]:idx_ppg_peak[k+1]+1], n/len_wave*len_interval_ppg) for n in range(len_wave)])

        for k in range(len(bool_noise_ecg)):
            len_interval_ecg = idx_ecg_peak[k+1] - idx_ecg_peak[k]

            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ecg.append([linear_connection(seg_ecg[idx_ecg_peak[k]:idx_ecg_peak[k+1]+1], n/len_wave*len_interval_ecg) for n in range(len_wave)])


        # wave interval 사이 correlation 계산 - PPG
        mean_wave_ppg = np.nanmean(norm_seg_ppg, axis = 0)
        mean_wave_ppg = pd.DataFrame(mean_wave_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ppg = pd.DataFrame(norm_seg_ppg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ppg)):
            if np.corrcoef(norm_seg_ppg[k], mean_wave_ppg)[0,1] < 0.9:
                bool_noise_ppg[k] = True
        noise_ppg_perc = np.sum(bool_noise_ppg) / len(bool_noise_ppg)

        # wave interval 사이 correlation 계산 - ECG                
        mean_wave_ecg = np.nanmean(norm_seg_ecg, axis = 0)
        mean_wave_ecg = pd.DataFrame(mean_wave_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ecg = pd.DataFrame(norm_seg_ecg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ecg)):
            if np.corrcoef(norm_seg_ecg[k], mean_wave_ecg)[0,1] < 0.9:
                bool_noise_ecg[k] = True
        noise_ecg_perc = np.sum(bool_noise_ecg) / len(bool_noise_ecg)

        # segment의 noise 비율 정보
        noise_info = [noise_ppg_perc, noise_ecg_perc]

        # segment를 input으로 써도 되는지
        if nan_ppg_perc < 0.05 and nan_ecg_perc < 0.05 and nan_both_perc < 0.05 and noise_ppg_perc < 0.1 and noise_ecg_perc < 0.1:
            bool_pass = True
        else:
            bool_pass = False

        # 이 segment의 정보를 dataframe에 저장 - (전처리 성공여부, 전처리 nan 비율, 전처리 noise 비율, 통증 점수)
        arry = np.empty(1, dtype=object)
        arry[0] = [bool_pass, nan_info, noise_info, 0, 0]
        df_preprocess.loc[f_num-1,f'{i+1}'] = arry[0] #{'pass':bool_pass, 'nan_perc':nan_info, 'noise_perc':noise_info, 'tss':0, 'cisa':0}        
        print('preprocessing done...', end='')
        ##########################################################################


    for i in range(n_aug2):
        print('  segment', i+1, end='')
        start_idx = i*OVERLAP*SRATE # 500i
        end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000


        ### event input ###
        seg_ppg = vals['PPG'][start_idx:end_idx]
        seg_ecg = vals['ECG'][start_idx:end_idx]


        ## 1. 결측치 처리 ##              
        # df.isnull().sum() 하면 더 간단하게 가능하나 애초에 NRS에 해당하는 vital data가 120초 보다 짧은 경우
        nan_ppg_list = np.isnan(seg_ppg)
        nan_ecg_list = np.isnan(seg_ecg)
        nan_ppg_perc = np.sum(nan_ppg_list) / LEN_INPUT / SRATE
        nan_ecg_perc = np.sum(nan_ecg_list) / LEN_INPUT / SRATE

        # ECG, PPG 둘다 결측치인 부분
        nan_both_perc = 0
        for j in range(len(seg_ppg)):
            if nan_ppg_list[j] and  nan_ecg_list[j]:
                nan_both_perc += 1
        nan_both_perc /= (LEN_INPUT*SRATE)

        # segment의 결측치 비율 정보
        nan_info = [nan_ppg_perc, nan_ecg_perc, nan_both_perc]

        # 결측치가 많은 경우, noise 확인할 것도 없이 False -  이 경우의 noise_info는 -1로 처리
        if nan_ppg_perc > 0.05 or nan_ecg_perc > 0.05 or nan_both_perc > 0.05:
            df_preprocess.loc[f_num-1,str(i+n_aug+1)] = (False, nan_info, [-1, -1])
            print('too much missing data')
            continue


        ## 2. Noise 처리 ##
        # peak detection
        if os.path.exists(ppg_cache+'_{}'.format(i+1)):
            _, ppg_peak = pickle.load(open(ppg_cache+'_{}'.format(i+1), 'rb'))
            ecg_peak = pickle.load(open(ecg_cache+'_{}'.format(i+1), 'rb'))
            print('...loaded peak...', end='')


        else:
            try:
                min_peak, ppg_peak = arr.detect_peaks(pd.DataFrame(seg_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)
                ecg_peak = arr.detect_qrs(pd.DataFrame(seg_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)


            except Exception as e:
                print('error of', e)
                error_list.append(caseid)
                df_preprocess.loc[f_num-1,str(i+n_aug+1)] = (False, nan_info, [-3, -3])
                continue


            if len(ppg_peak)==0:
                print('no peak')


            pickle.dump((min_peak, ppg_peak), open(ppg_cache+'_{}'.format(i+1), 'wb'))
            pickle.dump(ecg_peak, open(ecg_cache+'_{}'.format(i+1), 'wb'))
            print('...saved peak...', end='')


        # 10초 segment 내의 ppg, ecg peak idx
        #seg_ppg_min = ppg_min[(start_idx<=np.array(ppg_min)) & (np.array(ppg_min)<end_idx)]
        idx_ppg_peak = ppg_peak
        idx_ecg_peak = ecg_peak


        # peak가 HR 30~150 -> 20s - min 10 peaks(HR30)
        # peak 개수가 기준 미달이면 noise 계산 자세히 할 필요없이 False - 이 경우의 noise_info는 -2로 처리
        if len(idx_ppg_peak)<5/10*LEN_INPUT or len(idx_ecg_peak)<5/10*LEN_INPUT:
            df_preprocess.loc[f_num-1,str(i+n_aug+1)] = (False, nan_info, [-2, -2])
            print('too less peaks')
            continue


        # 20초 segment 내의 ppg, ecg peak value
        #print(len(seg_ppg), idx_ppg_peak)
        val_ppg_peak = [seg_ppg[k] for k in idx_ppg_peak]
        val_ecg_peak = [seg_ecg[k] for k in idx_ecg_peak]

        # peak와 peak 사이 interval에 대한 noise 여부 -> 따라서 길이는 peak - 1
        bool_noise_ppg = [False for k in range(len(idx_ppg_peak)-1)]
        bool_noise_ecg = [False for k in range(len(idx_ecg_peak)-1)]


        #  2.1 peak 간격 이상한 noise (HR 30~150 -> HBI 0.4s ~ 2s로 SRATE 곱해주면 40~200)
        for k in range(len(bool_noise_ppg)):
            if not 0.4*SRATE < idx_ppg_peak[k+1] - idx_ppg_peak[k] < 2*SRATE:
                bool_noise_ppg[k] = True
        for k in range(len(bool_noise_ecg)):
            if not 0.4*SRATE < idx_ecg_peak[k+1] - idx_ecg_peak[k] < 2*SRATE:
                bool_noise_ecg[k] = True


        # 2.2 모양 이상한 noise
        # wave interval into same length(2s(200))
        len_wave = 2*SRATE
        norm_seg_ppg, norm_seg_ecg = [], []

        for k in range(len(bool_noise_ppg)):
            len_interval_ppg = idx_ppg_peak[k+1] - idx_ppg_peak[k]

            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ppg.append([linear_connection(seg_ppg[idx_ppg_peak[k]:idx_ppg_peak[k+1]+1], n/len_wave*len_interval_ppg) for n in range(len_wave)])

        for k in range(len(bool_noise_ecg)):
            len_interval_ecg = idx_ecg_peak[k+1] - idx_ecg_peak[k]

            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ecg.append([linear_connection(seg_ecg[idx_ecg_peak[k]:idx_ecg_peak[k+1]+1], n/len_wave*len_interval_ecg) for n in range(len_wave)])


        # wave interval 사이 correlation 계산 - PPG
        mean_wave_ppg = np.nanmean(norm_seg_ppg, axis = 0)
        mean_wave_ppg = pd.DataFrame(mean_wave_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ppg = pd.DataFrame(norm_seg_ppg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ppg)):
            if np.corrcoef(norm_seg_ppg[k], mean_wave_ppg)[0,1] < 0.9:
                bool_noise_ppg[k] = True
        noise_ppg_perc = np.sum(bool_noise_ppg) / len(bool_noise_ppg)


        # wave interval 사이 correlation 계산 - ECG                
        mean_wave_ecg = np.nanmean(norm_seg_ecg, axis = 0)
        mean_wave_ecg = pd.DataFrame(mean_wave_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ecg = pd.DataFrame(norm_seg_ecg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ecg)):
            if np.corrcoef(norm_seg_ecg[k], mean_wave_ecg)[0,1] < 0.9:
                bool_noise_ecg[k] = True
        noise_ecg_perc = np.sum(bool_noise_ecg) / len(bool_noise_ecg)


        # segment의 noise 비율 정보
        noise_info = [noise_ppg_perc, noise_ecg_perc]

        # segment를 input으로 써도 되는지
        if nan_ppg_perc < 0.05 and nan_ecg_perc < 0.05 and nan_both_perc < 0.05 and noise_ppg_perc < 0.1 and noise_ecg_perc < 0.1:
            bool_pass = True
        else:
            bool_pass = False


        # 통증 점수 계산
        ### TSS(total surgical stimulation) = 1.57 - rftn20_ce / 3
        ### CISA(combined index of stimulus and analgesia) = stim_intensity - beta * ce + gamma, beta = 1/8, gamma = 1.5, stim_intensity = 5.5 
        rftn = vals['RFTN'][start_idx:end_idx]
        rftn = np.mean(rftn[~np.isnan(rftn)])
        tss = 1.57 - rftn / 3
        if tss < 0:
            tss = 0
        cisa = 7 - rftn / 8

        # 이 segment의 정보를 dataframe에 저장
        arry = np.empty(1, dtype=object)
        arry[0] = [bool_pass, nan_info, noise_info, tss, cisa]
        df_preprocess.loc[f_num-1,f'{i+n_aug+1}'] = arry[0] #{'pass':bool_pass, 'nan_perc':nan_info, 'noise_perc':noise_info, 'tss':0, 'cisa':0}            
        print('preprocessing done...', end='')            




###Input 1 / 1: 3###
  segment 1...loaded peak...preprocessing done...  segment 2...loaded peak...

/home/painstudy/.local/lib/python3.8/site-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


preprocessing done...  segment 3...loaded peak...preprocessing done...  segment 4...loaded peak...preprocessing done...  segment 5...loaded peak...preprocessing done...  segment 1...loaded peak...preprocessing done...  segment 2...loaded peak...preprocessing done...  segment 3...loaded peak...preprocessing done...  segment 4...loaded peak...preprocessing done...  segment 5...loaded peak...preprocessing done...  segment 6...loaded peak...preprocessing done...  segment 7...loaded peak...preprocessing done...  segment 8...loaded peak...preprocessing done...  segment 9...loaded peak...preprocessing done...  segment 10...loaded peak...preprocessing done...  segment 11...loaded peak...preprocessing done...

## input

In [35]:
# vital data 불러오기    
vals = np.load(f'{file_path}/{caseid}.npz')

x_train, tss_train, cisa_train = [], [], []
x_test, tss_test, cisa_test = [], [], []
x_val, tss_val, cisa_val = [], [], []
age_train, gender_train = [], []
age_test, gender_test = [], []
age_val, gender_val = [], []


caseid = '3'
row = df_preprocess.loc[0]
# 20초 단위로 끊기
for i in range(n_aug):
    print('  n_segment', i+1, end='')
    start_idx = i*OVERLAP*SRATE # 500i
    end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000

    # non-event data
    if row[str(i+1)][0]:
        print(' passed...lowess filtering...', end='')

        #save_path = f'cache/lowess_filtered/intu120s-input20s-10s/{caseid}_n{i+1}.npz'

        ppg_inp = vals['nPPG'][start_idx:end_idx]
        ecg_inp = vals['nECG'][start_idx:end_idx]

        ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()

        # lowess filter 적용
        ppg_input = ppg_inp - lowess(ppg_inp)
        ecg_input = ecg_inp - lowess(ecg_inp)

        ppg_input = ppg_input - np.nanmean(ppg_input)
        ecg_input = ecg_input - np.nanmean(ecg_input)
        #ecg_input = (ecg_input - min(ecg_input)) / (max(ecg_input) - min(ecg_input))
      
        
        x_test.append([ppg_input, ecg_input])
        tss_test.append(row[str(i+1)][3])
        cisa_test.append(row[str(i+1)][4])

        print('done', end=' ')

print('')
for i in range(n_aug2):
    print('  segment', i+1, end='')
    start_idx = i*OVERLAP*SRATE # 500i
    end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000

    # event data
    if row[str(i+n_aug+1)][0]:
        print(' passed...lowess filtering...', end='')

        #save_path = f'cache/lowess_filtered/input20s-10s/{caseid}_{i+1}.npz'
        ppg_inp = vals['PPG'][start_idx:end_idx]
        ecg_inp = vals['ECG'][start_idx:end_idx]

        ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()

        # lowess filter 적용
        ppg_input = ppg_inp - lowess(ppg_inp)
        ecg_input = ecg_inp - lowess(ecg_inp)

        ppg_input = ppg_input - np.nanmean(ppg_input)
        ecg_input = (ecg_input - min(ecg_input)) / (max(ecg_input) - min(ecg_input))


        x_test.append([ppg_input, ecg_input])
        tss_test.append(row[str(i+n_aug+1)][3])
        cisa_test.append(row[str(i+n_aug+1)][4])

         

  n_segment 1 passed...lowess filtering...done   n_segment 2 passed...lowess filtering...done   n_segment 3  n_segment 4  n_segment 5
  segment 1  segment 2  segment 3  segment 4  segment 5 passed...lowess filtering...  segment 6 passed...lowess filtering...  segment 7 passed...lowess filtering...  segment 8 passed...lowess filtering...  segment 9  segment 10  segment 11

In [34]:
df_preprocess

,caseid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,3,"[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[False, [0.0, 0.0, 0.0], [0.0, 0.1], 0, 0]","[False, [0.0, 0.0, 0.0], [0.0, 0.1666666666666...","[False, [0.0, 0.0, 0.0], [0.0, 0.15], 0, 0]","[False, [0.0, 0.0, 0.0], [0.12, 0.52], 0, 0]","[False, [0.0, 0.0, 0.0], [0.0, 0.7692307692307...","[False, [0.0, 0.0, 0.0], [0.0, 0.9615384615384...","[False, [0.0, 0.0, 0.0], [0.0, 0.96], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.043478260869565216,...","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[False, [0.0, 0.0, 0.0], [0.0, 0.2272727272727...","[False, [0.0, 0.0, 0.0], [0.0, 0.2173913043478...","[False, [0.0, 0.0, 0.0], [0.0, 0.1739130434782..."


In [45]:
a = np.empty(1, dtype=object)
a[0] = [bool_pass, nan_info, noise_info, 0,0]
df_preprocess.loc[f_num-1,f'{i+1}'] = a
df_preprocess

/home/painstudy/.local/lib/python3.8/site-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


,caseid,1,2,3,4,5,6,7,8,9,10,file_path
0,NaN,"[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5974


In [57]:
import shutil
shutil.rmtree('vital_to_np_60s')

In [ ]:
 # 피크 사이 wave를 모두 같은 length로 만들기 위한 함수
def linear_connection(list, idx):
    int_idx = int(idx)
    return list[int_idx] + (list[int_idx+1] - list[int_idx]) * (idx - int_idx)


file_path = 'vital_to_np'

# path for cache
if not os.path.exists('./cache'):
    os.mkdir('./cache')
if not os.path.exists('./cache/peaks'):
    os.mkdir('./cache/peaks')
if not os.path.exists(f"cache/peaks/PPG_{SRATE}Hz_1min_seg"):
    os.mkdir(f"cache/peaks/PPG_{SRATE}Hz_1min_seg")
if not os.path.exists(f"cache/peaks/ECG_{SRATE}Hz_1min_seg"):
    os.mkdir(f"cache/peaks/ECG_{SRATE}Hz_1min_seg")        
if not os.path.exists('./cache/preprocess'):
    os.mkdir('./cache/preprocess')


# dataframe to save preprocessing info
n_aug = int((LEN_PER_PRE-LEN_INPUT)/OVERLAP) + 1   # number of data augmentation
n_aug2 = int((LEN_PER_POST-LEN_INPUT)/OVERLAP) + 1
column_list = ['caseid'] + [str(i+1) for i in range(n_aug+n_aug2)]
df_preprocess = pd.DataFrame(columns = column_list)


# set variables
caseids = os.listdir(file_path)
error_list = []
f_num = 0
initial = f_num
interval = len(caseids)


for caseid in caseids[initial:initial+interval]:
    caseid = caseid[:-4]
    f_num += 1
    print('\n###Input', f_num,'/ '+str(len(caseids))+': '+caseid+'###')


    # vital data 불러오기    
    vals = np.load(f'{file_path}/{caseid}.npz')


    #dataframe에 새로운 행 만들기
    df_preprocess.loc[f_num-1,'caseid'] = caseid

    ppg_cache = f"cache/peaks/PPG_{SRATE}Hz_1min_seg/" + caseid
    ecg_cache = f"cache/peaks/ECG_{SRATE}Hz_1min_seg/" + caseid    
    ecg_cache2 = f"cache/peaks/ECG_{SRATE}Hz_1min_seg/" + caseid


    # 20초 단위로 끊기
    for i in range(n_aug):
        print('  segment', i+1, end='')
        start_idx = i*OVERLAP*SRATE # 500i
        end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000


        ### non-event input ###
        seg_ppg = vals['nPPG'][start_idx:end_idx]
        seg_ecg = vals['nECG'][start_idx:end_idx]


        ## 1. 결측치 처리 ##             
        # df.isnull().sum() 하면 더 간단하게 가능하나 애초에 NRS에 해당하는 vital data가 120초 보다 짧은 경우
        nan_ppg_list = np.isnan(seg_ppg)
        nan_ecg_list = np.isnan(seg_ecg)
        nan_ppg_perc = np.sum(nan_ppg_list) / LEN_INPUT / SRATE
        nan_ecg_perc = np.sum(nan_ecg_list) / LEN_INPUT / SRATE

        # ECG, PPG 둘다 결측치인 부분
        nan_both_perc = 0
        for j in range(len(seg_ppg)):
            if nan_ppg_list[j] and  nan_ecg_list[j]:
                nan_both_perc += 1
        nan_both_perc /= (LEN_INPUT*SRATE)

        # segment의 결측치 비율 정보
        nan_info = [nan_ppg_perc, nan_ecg_perc, nan_both_perc]

        # 결측치가 많은 경우, noise 확인할 것도 없이 False -  이 경우의 noise_info는 -1로 처리
        if nan_ppg_perc > 0.05 or nan_ecg_perc > 0.05 or nan_both_perc > 0.05:
            df_preprocess.loc[f_num-1,str(i+1)] = (False, nan_info, [-1, -1])
            print('too much missing data')
            continue


        ## 2. Noise 처리 ##
        # peak detection
        if os.path.exists(ppg_cache+'_n{}'.format(i+1)):
            _, ppg_peak = pickle.load(open(ppg_cache+'_n{}'.format(i+1), 'rb'))
            ecg_peak = pickle.load(open(ecg_cache+'_n{}'.format(i+1), 'rb'))
            print('...loaded peak...', end='')


        else:
            try:
                min_peak, ppg_peak = arr.detect_peaks(pd.DataFrame(seg_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)
                ecg_peak = arr.detect_qrs(pd.DataFrame(seg_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)


            except Exception as e:
                print('error of', e)
                error_list.append(caseid)
                df_preprocess.loc[f_num-1,str(i+1)] = (False, nan_info, [-3, -3])
                continue


            if len(ppg_peak)==0:
                print('no peak')


            pickle.dump((min_peak, ppg_peak), open(ppg_cache+'_n{}'.format(i+1), 'wb'))
            pickle.dump(ecg_peak, open(ecg_cache+'_n{}'.format(i+1), 'wb'))
            print('...saved peak...', end='')


        # 10초 segment 내의 ppg, ecg peak idx
        #seg_ppg_min = ppg_min[(start_idx<=np.array(ppg_min)) & (np.array(ppg_min)<end_idx)]
        idx_ppg_peak = ppg_peak
        idx_ecg_peak = ecg_peak


        # peak가 HR 30~150 -> 20s - min 10 peaks(HR30)
        # peak 개수가 기준 미달이면 noise 계산 자세히 할 필요없이 False - 이 경우의 noise_info는 -2로 처리
        if len(idx_ppg_peak)<5/10*LEN_INPUT or len(idx_ecg_peak)<5/10*LEN_INPUT:
            df_preprocess.loc[f_num-1,str(i+1)] = (False, nan_info, [-2, -2])
            print('too less peaks')
            continue


        # 20초 segment 내의 ppg, ecg peak value
        #print(len(seg_ppg), idx_ppg_peak)
        val_ppg_peak = [seg_ppg[k] for k in idx_ppg_peak]
        val_ecg_peak = [seg_ecg[k] for k in idx_ecg_peak]

        # peak와 peak 사이 interval에 대한 noise 여부 -> 따라서 길이는 peak - 1
        bool_noise_ppg = [False for k in range(len(idx_ppg_peak)-1)]
        bool_noise_ecg = [False for k in range(len(idx_ecg_peak)-1)]


        #  2.1 peak 간격 이상한 noise (HR 30~150 -> HBI 0.4s ~ 2s로 SRATE 곱해주면 40~200)
        for k in range(len(bool_noise_ppg)):
            if not 0.4*SRATE < idx_ppg_peak[k+1] - idx_ppg_peak[k] < 2*SRATE:
                bool_noise_ppg[k] = True
        for k in range(len(bool_noise_ecg)):
            if not 0.4*SRATE < idx_ecg_peak[k+1] - idx_ecg_peak[k] < 2*SRATE:
                bool_noise_ecg[k] = True


        # 2.2 모양 이상한 noise
        # wave interval into same length(2s(200))
        len_wave = 2*SRATE
        norm_seg_ppg, norm_seg_ecg = [], []

        for k in range(len(bool_noise_ppg)):
            len_interval_ppg = idx_ppg_peak[k+1] - idx_ppg_peak[k]

            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ppg.append([linear_connection(seg_ppg[idx_ppg_peak[k]:idx_ppg_peak[k+1]+1], n/len_wave*len_interval_ppg) for n in range(len_wave)])

        for k in range(len(bool_noise_ecg)):
            len_interval_ecg = idx_ecg_peak[k+1] - idx_ecg_peak[k]

            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ecg.append([linear_connection(seg_ecg[idx_ecg_peak[k]:idx_ecg_peak[k+1]+1], n/len_wave*len_interval_ecg) for n in range(len_wave)])


        # wave interval 사이 correlation 계산 - PPG
        mean_wave_ppg = np.nanmean(norm_seg_ppg, axis = 0)
        mean_wave_ppg = pd.DataFrame(mean_wave_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ppg = pd.DataFrame(norm_seg_ppg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ppg)):
            if np.corrcoef(norm_seg_ppg[k], mean_wave_ppg)[0,1] < 0.9:
                bool_noise_ppg[k] = True
        noise_ppg_perc = np.sum(bool_noise_ppg) / len(bool_noise_ppg)

        # wave interval 사이 correlation 계산 - ECG                
        mean_wave_ecg = np.nanmean(norm_seg_ecg, axis = 0)
        mean_wave_ecg = pd.DataFrame(mean_wave_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ecg = pd.DataFrame(norm_seg_ecg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ecg)):
            if np.corrcoef(norm_seg_ecg[k], mean_wave_ecg)[0,1] < 0.9:
                bool_noise_ecg[k] = True
        noise_ecg_perc = np.sum(bool_noise_ecg) / len(bool_noise_ecg)

        # segment의 noise 비율 정보
        noise_info = [noise_ppg_perc, noise_ecg_perc]

        # segment를 input으로 써도 되는지
        if nan_ppg_perc < 0.05 and nan_ecg_perc < 0.05 and nan_both_perc < 0.05 and noise_ppg_perc < 0.1 and noise_ecg_perc < 0.1:
            bool_pass = True
        else:
            bool_pass = False

        # 이 segment의 정보를 dataframe에 저장 - (전처리 성공여부, 전처리 nan 비율, 전처리 noise 비율, 통증 점수)
        arry = np.empty(1, dtype=object)
        arry[0] = [bool_pass, nan_info, noise_info, 0, 0]
        df_preprocess.loc[f_num-1,f'{i+1}'] = arry[0] #{'pass':bool_pass, 'nan_perc':nan_info, 'noise_perc':noise_info, 'tss':0, 'cisa':0}        
        print('preprocessing done...', end='')
        ##########################################################################


    for i in range(n_aug2):
        print('  segment', i+1, end='')
        start_idx = i*OVERLAP*SRATE # 500i
        end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000


        ### event input ###
        seg_ppg = vals['PPG'][start_idx:end_idx]
        seg_ecg = vals['ECG'][start_idx:end_idx]


        ## 1. 결측치 처리 ##              
        # df.isnull().sum() 하면 더 간단하게 가능하나 애초에 NRS에 해당하는 vital data가 120초 보다 짧은 경우
        nan_ppg_list = np.isnan(seg_ppg)
        nan_ecg_list = np.isnan(seg_ecg)
        nan_ppg_perc = np.sum(nan_ppg_list) / LEN_INPUT / SRATE
        nan_ecg_perc = np.sum(nan_ecg_list) / LEN_INPUT / SRATE

        # ECG, PPG 둘다 결측치인 부분
        nan_both_perc = 0
        for j in range(len(seg_ppg)):
            if nan_ppg_list[j] and  nan_ecg_list[j]:
                nan_both_perc += 1
        nan_both_perc /= (LEN_INPUT*SRATE)

        # segment의 결측치 비율 정보
        nan_info = [nan_ppg_perc, nan_ecg_perc, nan_both_perc]

        # 결측치가 많은 경우, noise 확인할 것도 없이 False -  이 경우의 noise_info는 -1로 처리
        if nan_ppg_perc > 0.05 or nan_ecg_perc > 0.05 or nan_both_perc > 0.05:
            df_preprocess.loc[f_num-1,str(i+n_aug+1)] = (False, nan_info, [-1, -1])
            print('too much missing data')
            continue


        ## 2. Noise 처리 ##
        # peak detection
        if os.path.exists(ppg_cache+'_{}'.format(i+1)):
            _, ppg_peak = pickle.load(open(ppg_cache+'_{}'.format(i+1), 'rb'))
            ecg_peak = pickle.load(open(ecg_cache+'_{}'.format(i+1), 'rb'))
            print('...loaded peak...', end='')


        else:
            try:
                min_peak, ppg_peak = arr.detect_peaks(pd.DataFrame(seg_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)
                ecg_peak = arr.detect_qrs(pd.DataFrame(seg_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)


            except Exception as e:
                print('error of', e)
                error_list.append(caseid)
                df_preprocess.loc[f_num-1,str(i+n_aug+1)] = (False, nan_info, [-3, -3])
                continue


            if len(ppg_peak)==0:
                print('no peak')


            pickle.dump((min_peak, ppg_peak), open(ppg_cache+'_{}'.format(i+1), 'wb'))
            pickle.dump(ecg_peak, open(ecg_cache+'_{}'.format(i+1), 'wb'))
            print('...saved peak...', end='')


        # 10초 segment 내의 ppg, ecg peak idx
        #seg_ppg_min = ppg_min[(start_idx<=np.array(ppg_min)) & (np.array(ppg_min)<end_idx)]
        idx_ppg_peak = ppg_peak
        idx_ecg_peak = ecg_peak


        # peak가 HR 30~150 -> 20s - min 10 peaks(HR30)
        # peak 개수가 기준 미달이면 noise 계산 자세히 할 필요없이 False - 이 경우의 noise_info는 -2로 처리
        if len(idx_ppg_peak)<5/10*LEN_INPUT or len(idx_ecg_peak)<5/10*LEN_INPUT:
            df_preprocess.loc[f_num-1,str(i+n_aug+1)] = (False, nan_info, [-2, -2])
            print('too less peaks')
            continue


        # 20초 segment 내의 ppg, ecg peak value
        #print(len(seg_ppg), idx_ppg_peak)
        val_ppg_peak = [seg_ppg[k] for k in idx_ppg_peak]
        val_ecg_peak = [seg_ecg[k] for k in idx_ecg_peak]

        # peak와 peak 사이 interval에 대한 noise 여부 -> 따라서 길이는 peak - 1
        bool_noise_ppg = [False for k in range(len(idx_ppg_peak)-1)]
        bool_noise_ecg = [False for k in range(len(idx_ecg_peak)-1)]


        #  2.1 peak 간격 이상한 noise (HR 30~150 -> HBI 0.4s ~ 2s로 SRATE 곱해주면 40~200)
        for k in range(len(bool_noise_ppg)):
            if not 0.4*SRATE < idx_ppg_peak[k+1] - idx_ppg_peak[k] < 2*SRATE:
                bool_noise_ppg[k] = True
        for k in range(len(bool_noise_ecg)):
            if not 0.4*SRATE < idx_ecg_peak[k+1] - idx_ecg_peak[k] < 2*SRATE:
                bool_noise_ecg[k] = True


        # 2.2 모양 이상한 noise
        # wave interval into same length(2s(200))
        len_wave = 2*SRATE
        norm_seg_ppg, norm_seg_ecg = [], []

        for k in range(len(bool_noise_ppg)):
            len_interval_ppg = idx_ppg_peak[k+1] - idx_ppg_peak[k]

            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ppg.append([linear_connection(seg_ppg[idx_ppg_peak[k]:idx_ppg_peak[k+1]+1], n/len_wave*len_interval_ppg) for n in range(len_wave)])

        for k in range(len(bool_noise_ecg)):
            len_interval_ecg = idx_ecg_peak[k+1] - idx_ecg_peak[k]

            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ecg.append([linear_connection(seg_ecg[idx_ecg_peak[k]:idx_ecg_peak[k+1]+1], n/len_wave*len_interval_ecg) for n in range(len_wave)])


        # wave interval 사이 correlation 계산 - PPG
        mean_wave_ppg = np.nanmean(norm_seg_ppg, axis = 0)
        mean_wave_ppg = pd.DataFrame(mean_wave_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ppg = pd.DataFrame(norm_seg_ppg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ppg)):
            if np.corrcoef(norm_seg_ppg[k], mean_wave_ppg)[0,1] < 0.9:
                bool_noise_ppg[k] = True
        noise_ppg_perc = np.sum(bool_noise_ppg) / len(bool_noise_ppg)


        # wave interval 사이 correlation 계산 - ECG                
        mean_wave_ecg = np.nanmean(norm_seg_ecg, axis = 0)
        mean_wave_ecg = pd.DataFrame(mean_wave_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ecg = pd.DataFrame(norm_seg_ecg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ecg)):
            if np.corrcoef(norm_seg_ecg[k], mean_wave_ecg)[0,1] < 0.9:
                bool_noise_ecg[k] = True
        noise_ecg_perc = np.sum(bool_noise_ecg) / len(bool_noise_ecg)


        # segment의 noise 비율 정보
        noise_info = [noise_ppg_perc, noise_ecg_perc]

        # segment를 input으로 써도 되는지
        if nan_ppg_perc < 0.05 and nan_ecg_perc < 0.05 and nan_both_perc < 0.05 and noise_ppg_perc < 0.1 and noise_ecg_perc < 0.1:
            bool_pass = True
        else:
            bool_pass = False


        # 통증 점수 계산
        ### TSS(total surgical stimulation) = 1.57 - rftn20_ce / 3
        ### CISA(combined index of stimulus and analgesia) = stim_intensity - beta * ce + gamma, beta = 1/8, gamma = 1.5, stim_intensity = 5.5 
        rftn = vals['RFTN'][start_idx:end_idx]
        rftn = np.mean(rftn[~np.isnan(rftn)])
        tss = 1.57 - rftn / 3
        if tss < 0:
            tss = 0
        cisa = 7 - rftn / 8

        # 이 segment의 정보를 dataframe에 저장
        arry = np.empty(1, dtype=object)
        arry[0] = [bool_pass, nan_info, noise_info, tss, cisa]
        df_preprocess.loc[f_num-1,f'{i+n_aug+1}'] = arry[0] #{'pass':bool_pass, 'nan_perc':nan_info, 'noise_perc':noise_info, 'tss':0, 'cisa':0}            
        print('preprocessing done...', end='')            

print(f'\ndumping cache of df_preprocess {f_num}/{len(caseids)}', end='...')

# df_preprocess에 demographs(age, gender) 추가
df_demograph = pd.read_csv("https://api.vitaldb.net/cases")
df_preprocess['age'] = np.nan
df_preprocess['gender'] = np.nan

for idx, row in df_preprocess.iterrows():     
    row_demo = df_demograph[df_demograph['caseid']==int(row['caseid'])]

    df_preprocess.loc[idx, 'age'] = row_demo['age'].values[0]
    df_preprocess.loc[idx, 'gender'] = row_demo['sex'].values[0]

df_preprocess.reset_index(drop=True, inplace=True)    
pickle.dump(df_preprocess, open('cache/preprocess/df_preprocess', 'wb'))
print('dumping success')

# 전처리 통과 비율 출력
ne_pass, e_pass = 0, 0

for _, row in df_preprocess.iterrows():   
    for i in range(n_aug):
        if row[str(i+1)][0]:
            ne_pass = ne_pass + 1

    for i in range(n_aug, n_aug+n_aug2):
        if row[str(i+1)][0]:
            e_pass = e_pass + 1

print(f'non-event seg pass: {ne_pass/n_aug/2684*100:.2f}%, event seg pass: {e_pass/n_aug2/2684*100:.2f}%')
print(f'passed segments : {ne_pass+e_pass}')

## Learn.py (2022.01.15)

In [ ]:
%load_ext autoreload
%autoreload 2

import learn

learn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
loading vital data...
1/2699(3)...48557   completed
2/2699(8)...11997   completed
3/2699(9)...39190   completed
4/2699(13)...86596   completed
5/2699(14)...35138   completed
6/2699(16)...36578   completed
7/2699(17)...36990   completed
8/2699(19)...40770   completed
9/2699(20)...70645   completed
10/2699(22)...28427   completed
11/2699(28)...47418   completed
12/2699(29)...97266   completed
13/2699(30)...23567   completed
14/2699(35)...61530   completed
15/2699(36)...40082   completed
16/2699(40)...38947   completed
17/2699(42)...84053   completed
18/2699(44)...42878   completed
19/2699(45)...91473   completed
20/2699(46)...82007   completed
21/2699(47)...21423   completed
22/2699(48)...123143   completed
23/2699(50)...30938   completed
24/2699(54)...29642   completed
25/2699(57)...30190   completed
26/2699(59)...44006   completed
27/2699(67)...178394   completed
28/2699(69)...45849   completed
29/2

/home/painstudy/.local/lib/python3.8/site-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


...saved peak...preprocessing done...  segment 3...saved peak...preprocessing done...  segment 4...saved peak...preprocessing done...  segment 5...saved peak...preprocessing done...  segment 1...saved peak...preprocessing done...  segment 2...saved peak...preprocessing done...  segment 3...saved peak...preprocessing done...  segment 4...saved peak...preprocessing done...  segment 5...saved peak...preprocessing done...  segment 6...saved peak...preprocessing done...  segment 7...saved peak...preprocessing done...  segment 8...saved peak...preprocessing done...  segment 9...saved peak...preprocessing done...  segment 10...saved peak...preprocessing done...  segment 11...saved peak...preprocessing done...
###Input 2 / 2684: 801###
  segment 1...saved peak...preprocessing done...  segment 2...saved peak...preprocessing done...  segment 3...saved peak...preprocessing done...  segment 4...saved peak...preprocessing done...  segment 5...saved peak...preprocessing done...  segment 1...saved pe

/home/painstudy/anaconda3/envs/keras/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/painstudy/anaconda3/envs/keras/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


...saved peak...preprocessing done...  segment 8...saved peak...preprocessing done...  segment 9...saved peak...preprocessing done...  segment 10...saved peak...preprocessing done...  segment 11...saved peak...preprocessing done...
###Input 322 / 2684: 1639###
  segment 1...saved peak...preprocessing done...  segment 2...saved peak...too less peaks
  segment 3...saved peak...too less peaks
  segment 4...saved peak...too less peaks
  segment 5...saved peak...too less peaks
  segment 1...saved peak...preprocessing done...  segment 2...saved peak...preprocessing done...  segment 3...saved peak...preprocessing done...  segment 4...saved peak...preprocessing done...  segment 5...saved peak...preprocessing done...  segment 6...saved peak...preprocessing done...  segment 7...saved peak...preprocessing done...  segment 8...saved peak...preprocessing done...  segment 9...saved peak...preprocessing done...  segment 10...saved peak...preprocessing done...  segment 11...saved peak...preprocessing 

In [49]:
BATCH_SIZE = 1024
MAX_CASES = 2000
SEGLEN_IN_SEC = 20
SRATE = 100
LEN_INPUT = 20
OVERLAP = 10
LEN_PER_PRE = 60
LEN_PER_POST = 120


# input - filtering, saving
input_path = f"../DL_model/dataset/ne{LEN_PER_PRE}s-e{LEN_PER_POST}s-len{LEN_INPUT}-{OVERLAP}"
if not os.path.exists('../DL_model/dataset'):
    os.mkdir('../DL_model/dataset')
if not os.path.exists(input_path):
    os.mkdir(input_path)

df_preprocess = pickle.load(open('../Preprocess/cache/preprocess/df_preprocess', 'rb'))
caseid_test = pickle.load(open('../DL_model/caseid_test', 'rb'))
caseid_train = pickle.load(open('../DL_model/caseid_train', 'rb'))
caseid_val = pickle.load(open('../DL_model/caseid_val', 'rb'))
    
    
    
# variables
non_lis = []
x_train, tss_train, cisa_train = [], [], []
x_test, tss_test, cisa_test = [], [], []
x_val, tss_val, cisa_val = [], [], []
age_train, gender_train = [], []
age_test, gender_test = [], []
age_val, gender_val = [], []
     

for f_num, row in df_preprocess.iterrows():
    caseid = row['caseid']
    print(f'\n###Input{f_num}/{len(df_preprocess)}: {caseid}###')
    

    # vital data 불러오기    
    vals = np.load(f'{file_path}/{caseid}.npz')


    # 20초 단위로 끊기
    for i in range(n_aug):
        print('  n_segment', i+1, end='')
        start_idx = i*OVERLAP*SRATE # 500i
        end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000
        
        # non-event data
        if row[str(i+1)][0]:
            print(' passed...lowess filtering...', end='')
            
            #save_path = f'cache/lowess_filtered/intu120s-input20s-10s/{caseid}_n{i+1}.npz'

            ppg_inp = vals['nPPG'][start_idx:end_idx]
            ecg_inp = vals['nECG'][start_idx:end_idx]
            
            ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
            ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
            
            # lowess filter 적용
            ppg_input = ppg_inp - lowess(ppg_inp)
            ecg_input = ecg_inp - lowess(ecg_inp)
            
            ppg_input = ppg_input - np.nanmean(ppg_input)
            ecg_input = ecg_input - np.nanmean(ecg_input)
            #ecg_input = (ecg_input - min(ecg_input)) / (max(ecg_input) - min(ecg_input))
            

            # 해당 caseid가 test set에 속하는 경우
            if row['caseid'] in caseid_test:
                age_test.append(int(row['age']))
                if row['gender']=='F':
                    gender_test.append(1)
                else:
                    gender_test.append(0)
                x_test.append([ppg_input, ecg_input])
                tss_test.append(row[str(i+1)][3])
                cisa_test.append(row[str(i+1)][4])

            # 해당 caseid가 val set에 해당하는 경우
            elif row['caseid'] in caseid_val:
                age_val.append(int(row['age']))
                if row['gender']=='F':
                    gender_val.append(1)
                else:
                    gender_val.append(0)                    
                x_val.append([ppg_input, ecg_input])
                tss_val.append(row[str(i+1)][3])
                cisa_val.append(row[str(i+1)][4])

            # 해당 caseid가 train set에 해당하는 경우
            elif row['caseid'] in caseid_train:
                age_train.append(int(row['age']))
                if row['gender']=='F':
                    gender_train.append(1)
                else:
                    gender_train.append(0)                    
                x_train.append([ppg_input, ecg_input])
                tss_train.append(row[str(i+1)][3])
                cisa_train.append(row[str(i+1)][4])

            else:
                print('no case%$')
                non_lis.append(row['caseid'])

            #np.savez(save_path, ECG = ecg_input, PPG = ppg_input)
            print('done', end=' ')
    
    print('')
    for i in range(n_aug2):
        print('  segment', i+1, end='')
        start_idx = i*OVERLAP*SRATE # 500i
        end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000
            
        # event data
        if row[str(i+n_aug+1)][0]:
            print(' passed...lowess filtering...', end='')
            
            #save_path = f'cache/lowess_filtered/input20s-10s/{caseid}_{i+1}.npz'
            ppg_inp = vals['PPG'][start_idx:end_idx]
            ecg_inp = vals['ECG'][start_idx:end_idx]
            
            ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
            ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
            
            # lowess filter 적용
            ppg_input = ppg_inp - lowess(ppg_inp)
            ecg_input = ecg_inp - lowess(ecg_inp)
            
            ppg_input = ppg_input - np.nanmean(ppg_input)
            ecg_input = (ecg_input - min(ecg_input)) / (max(ecg_input) - min(ecg_input))


            # 해당 caseid가 test set에 속하는 경우
            if row['caseid'] in caseid_test:
                age_test.append(int(row['age']))
                if row['gender']=='F':
                    gender_test.append(1)
                else:
                    gender_test.append(0)
                x_test.append([ppg_input, ecg_input])
                tss_test.append(row[str(i+n_aug+1)][3])
                cisa_test.append(row[str(i+n_aug+1)][4])

            # 해당 caseid가 val set에 해당하는 경우
            elif row['caseid'] in caseid_val:
                age_val.append(int(row['age']))
                if row['gender']=='F':
                    gender_val.append(1)
                else:
                    gender_val.append(0)                    
                x_val.append([ppg_input, ecg_input])
                tss_val.append(row[str(i+n_aug+1)][3])
                cisa_val.append(row[str(i+n_aug+1)][4])

            # 해당 caseid가 train set에 해당하는 경우
            elif row['caseid'] in caseid_train:
                age_train.append(int(row['age']))
                if row['gender']=='F':
                    gender_train.append(1)
                else:
                    gender_train.append(0)                    
                x_train.append([ppg_input, ecg_input])
                tss_train.append(row[str(i+n_aug+1)][3])
                cisa_train.append(row[str(i+n_aug+1)][4])

            else:
                print('no case%$')
                non_lis.append(row['caseid'])                 
            
            #np.savez(save_path, ECG = ecg_input, PPG = ppg_input) 
            print('done', end=' ')

x_train = np.array(x_train, np.float32)
x_test = np.array(x_test, np.float32)
x_val = np.array(x_val, np.float32)
tss_train = np.array(tss_train, np.float32)
tss_test = np.array(tss_test, np.float32)
tss_val = np.array(tss_val, np.float32)
cisa_train = np.array(cisa_train, np.float32)
cisa_test = np.array(cisa_test, np.float32)
cisa_val = np.array(cisa_val, np.float32)

age_train = np.array(age_train, int)
age_test = np.array(age_test, int)
age_val = np.array(age_val, int)
gender_train = np.array(gender_train, int)
gender_test = np.array(gender_test, int)
gender_val = np.array(gender_val, int)


# 알맞게 input 변환
x_train = np.transpose(x_train, [0,2,1])
x_val = np.transpose(x_val, [0,2,1])
x_test = np.transpose(x_test, [0,2,1])

print('after concatenate + transpose')
print('x_train shape:', x_train.shape)
print('x_val shape:', x_val.shape)
print('x_test shape:', x_test.shape)


# 저장하기
print('saving...', end='', flush=True)
np.savez_compressed(input_path+'x_train.npz', x_train)
np.savez_compressed(input_path+'x_test.npz', x_test)
np.savez_compressed(input_path+'x_val.npz', x_val)
np.savez_compressed(input_path+'tss_train.npz', tss_train)
np.savez_compressed(input_path+'tss_test.npz', tss_test)
np.savez_compressed(input_path+'tss_val.npz', tss_val)
np.savez_compressed(input_path+'cisa_train.npz', cisa_train)
np.savez_compressed(input_path+'cisa_test.npz', cisa_test)
np.savez_compressed(input_path+'cisa_val.npz', cisa_val)

np.savez_compressed(input_path+'age_train.npz', age_train)
np.savez_compressed(input_path+'age_test.npz', age_test)
np.savez_compressed(input_path+'age_val.npz', age_val)    
np.savez_compressed(input_path+'gender_train.npz', gender_train)
np.savez_compressed(input_path+'gender_test.npz', gender_test)
np.savez_compressed(input_path+'gender_val.npz', gender_val)    

print('done', flush=True)
print('size of training set(pacu):', len(x_train))
print('size of validation set(pacu):', len(x_val))
print('size of test set(pacu):', len(x_test))


###Input0/2684: 5974###
  n_segment 1 passed...lowess filtering...done   n_segment 2 passed...lowess filtering...done   n_segment 3 passed...lowess filtering...done   n_segment 4 passed...lowess filtering...done   n_segment 5 passed...lowess filtering...done 
  segment 1 passed...lowess filtering...done   segment 2 passed...lowess filtering...done   segment 3 passed...lowess filtering...done   segment 4 passed...lowess filtering...done   segment 5 passed...lowess filtering...done   segment 6 passed...lowess filtering...done   segment 7 passed...lowess filtering...done   segment 8 passed...lowess filtering...done   segment 9 passed...lowess filtering...done   segment 10 passed...lowess filtering...done   segment 11 passed...lowess filtering...done 
###Input1/2684: 801###
  n_segment 1 passed...lowess filtering...done   n_segment 2 passed...lowess filtering...done   n_segment 3 passed...lowess filtering...done   n_segment 4 passed...lowess filtering...done   n_segment 5 passed...lowess 

In [54]:
df_preprocess

,caseid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,age,gender
0,5974,"[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.03846153846153...","[True, [0.0, 0.0, 0.0], [0.0, 0.04], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.038461538461538464,...","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]",49.0,F
1,801,"[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[False, [0.0, 0.0, 0.0], [0.03225806451612903,...","[False, [0.0, 0.0, 0.0], [0.03225806451612903,...","[False, [0.0, 0.0, 0.0], [0.09375, 0.892857142...","[False, [0.0, 0.0, 0.0], [0.23333333333333334,...","[False, [0.0, 0.0, 0.0], [0.3103448275862069, ...","[False, [0.0, 0.0, 0.0], [0.06451612903225806,...","[False, [0.0, 0.0, 0.0], [0.0, 0.2916666666666...","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[False, [0.0, 0.0, 0.0], [0.15384615384615385,...","[False, [0.0, 0.0, 0.0], [0.24, 0.296296296296...",62.0,F
2,2790,"[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.04], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.08], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.08], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.09523809523809...","[True, [0.0, 0.0, 0.0], [0.0, 0.09523809523809...",60.0,F
3,1413,"[True, [0.0, 0.0, 0.0], [0.0, 0.05], 0, 0]","[True, [0.0, 0.0, 0.0], [0.05, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.047619047619047616,...","[False, [0.0, 0.0, 0.0], [0.0, 0.1], 0, 0]","[False, [0.0, 0.0, 0.0], [0.0, 0.2], 0, 0]","[False, [0.0, 0.0, 0.0], [0.18181818181818182,...","[False, [0.0, 0.0, 0.0], [0.5, 0.1739130434782...","[False, [0.0, 0.0, 0.0], [0.42857142857142855,...","[False, [0.0, 0.0, 0.0], [0.5882352941176471, ...","(False, [0.0, 0.0, 0.0], [-2, -2])","(False, [0.0, 0.0, 0.0], [-2, -2])","[False, [0.0, 0.0, 0.0], [1.0, 0.0476190476190...","(False, [0.0, 0.0, 0.0], [-2, -2])","(False, [0.0, 0.0, 0.0], [-2, -2])","[False, [0.0, 0.0, 0.0], [1.0, 0.05], 0, 0]","[False, [0.0, 0.0, 0.0], [1.0, 0.0], 0, 0]",48.0,M
4,4057,"[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0625, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.058823529411764705,...","[True, [0.0, 0.0, 0.0], [0.058823529411764705,...","[False, [0.0, 0.0, 0.0], [0.058823529411764705...","[True, [0.0, 0.0, 0.0], [0.0, 0.05882352941176...","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]","[True, [0.0, 0.0, 0.0], [0.0, 0.0], 0, 0]",50.0,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2679,5882,"[False, [0.0, 0.0, 0.0], [0.0, 0.9285714285714...","[False, [0.0, 0.0, 0.0], [0.0, 0.6666666

In [52]:
df_preprocess = pickle.load(open('cache/preprocess/df_preprocess', 'rb'))

# 전처리 통과 비율 출력
ne_pass, e_pass = 0, 0

for _, row in df_preprocess.iterrows():   
    for i in range(n_aug):
        if row[str(i+1)][0]:
            ne_pass = ne_pass + 1

    for i in range(n_aug, n_aug+n_aug2):
        if row[str(i+1)][0]:
            e_pass = e_pass + 1

print(f'non-event seg pass: {ne_pass/5/2684*100:.2f}%, event seg pass: {e_pass/n_aug2/2684*100:.2f}%')
print(f'passed segments : {ne_pass+e_pass}')

# shuffle caseids which has survived preprocessing
p_caseids = []
for _, row in df_preprocess.iterrows():
    for i in range(0,10):
        if row[str(i+1)][0]:
            p_caseids.append(row['caseid'])
            break
print(f'survived caseids : {len(p_caseids)} cases / {len(df_preprocess)} cases')

non-event seg pass: 46.51%, event seg pass: 39.16%
passed segments : 17802
survived caseids : 2101 cases / 2684 cases
